# SEIR-model-Stockholm-Julia
I got tired of waiting for the R code to execute, so I decided to rewrite the initial parts of [the script](https://github.com/FohmAnalys/SEIR-model-Stockholm/commit/8c79ba30ea154ff47f1a9676a8f68c0d3b84793c#diff-824cda0829d07b1639809e1579eb586c) in Julia, with multithreading in order to speed things up. With a typical quad-core, the speed is roughly on the order of $10^{2.5}$ times that of the R code on the second run. Going from 30min to 10s allows for more convenient exploration. Note that the JIT-compilation will cause the first run to be significantly slower. Note that `@everywhere` is required for code to be run on all threads.

The solutions found are not always exactly the same as the R equivalent, but in some cases RSS (the loss function) is lower, which could indicate a better fit.

I do not guarantee that any of the code below will yield the same results as the original code. Use at your own risk.

## Imports

In [1]:
using Plots, DataFrames, Distributions;
using RData, FileIO, CSVFiles, ExcelFiles; # ExcelFiles relies on the Python xlrd package.
import Random.rand, CSV;
import Calculus.hessian, Statistics.mean, LinearAlgebra.Diagonal;

In [2]:
## Multithreading
using Distributed;

# Set the number of workers to equal the number of CPU threads.
if length(workers())>1 rmprocs(workers());end;addprocs();

# Import the following packages into the threads.
@everywhere using Optim, OrdinaryDiffEq, Dates, SharedArrays;

In [3]:
# Functions that make translation from R a bit easier...
@everywhere function runif(c::Int, min::Real, max::Real)::Vector{Float64}
    return rand(c) .* (max-min) .+ min;
    end;

# For quick date conversion
function dayToDate(day::Int)::Date
    return Dates.Day(day) + Dates.Date("2019-12-31");
    end;

@everywhere function dateToDay(date::Date)::Int
    return Dates.Day(date - Dates.Date("2019-12-31")).value;
    end;

In [4]:
## From a sample x, calculate a credibility interval with wanted level. 
function CRI(x::AbstractVector{Float64}; level::Float64 = 0.95)::Tuple{Float64, Float64}
    local n::Int, L::Float64, U::Float64, resL::Float64, resU::Float64;
    n = length(x);
    L = (1 - level) / 2;
    U = 1 - (1 - level) / 2;
    x = sort(x);
    #@ Added rounding here.
    resL = x[Int(round(n * L))];
    resU = x[Int(round(n * U))];
    #names(resL) <- paste((L * 100), "%") 
    #names(resU) <- paste((U * 100), "%")

    return (resL, resU);
    end;

function CRI_90_low(x::AbstractVector{Float64}) return CRI(x, level = 0.9)[1]; end;
function CRI_90_up(x::AbstractVector{Float64})  return CRI(x, level = 0.9)[2]; end;
function CRI_95_low(x::AbstractVector{Float64}) return CRI(x, level = 0.95)[1]; end;
function CRI_95_up(x::AbstractVector{Float64})  return CRI(x, level = 0.95)[2]; end;

## Load data
Currently just uses the default directories for loading.

In [5]:
begin
    local rd, rp, r_name, region_namn, df_riket;

    # Take out population
    rd = load(File(DataFormat{:RData},"Data/Sverige_population_2019.Rdata"))

    # rp is a DataFrame that is turned into Region_population
    rp          = rd["dat_pop_region_totalt"];
    rp[!, :Pop] = Int.(rp[!, :Pop]);
    r_name      = rp[!, :ARegion];

    region_namn = replace.(r_name, "s län" => "");
    region_namn = replace.(region_namn, " län" => "");
    region_namn = region_namn[region_namn .!= "Riket"];

    rp[!, :ARegion] = region_namn;

    df_riket = DataFrame(ARegion = "Riket", Pop = 2385128+5855459+2078886);
    
    global const Stockholm_Data = rename(CSV.read("Data/Data_2020-04-14Ny.txt", delim=" ", header = false), [:Datum, :Incidens]);
    global const Region_population = vcat(rp, df_riket);
end;

## Global variables and constants
A fair amount of variables that were local in the original code have been changed into global constants, as they shouldn't be changing. Some global vars are defined at analysis.

### Parameters
Feel free to redefine these at the analysis stage.

In [6]:
# Rate of leaving incubation
eta_value    = 1/5.1;
# Rate of recovery
gammaD_value = 1/5;

## Tolerance for ode and optimisation. 
Atol = 1e-8;
Rtol = 1e-10;

### Previously local variables
Constants that used to be a part of `Estimate_function_Stockholm`, and its subfunctions. Note that the variable called `Day` in the R script has been renamed to `Dag`, as not to conflict with the `Dates.Day` function.

In [7]:
# Population of Stockholm
const N = Region_population[Region_population[!, :ARegion] .== "Stockholm", 2][1];

## Daily incidence reported cases and their dates
const Incidence = Stockholm_Data[!, :Incidens];
const Datum = Date.(Stockholm_Data[!, :Datum]);
const Dag = dateToDay.(Datum);
# Used by the ode solver
const DagspanF = (Float64(minimum(Dag)), Float64(maximum(Dag)));

const dayatmonth = [1,31,29,31,30,31,30,31,31,30,31,30,31];
const dayatyear = [cumsum(dayatmonth)...];
const Namedate = [[Dates.Day(d) + Date("2019-12-31") for d in dayatyear]...];

const Opt_par_names = ["delta","epsilon","theta"];

## assumption on initial number of infected. 
## In our main analysis we start with 1 infectious individual at t_0 = 17th Ferbruary 
# init <- c(S = N - Incidence[1]*(1 + (1-p_symp)/p_symp), E = 0, I_symp = Incidence[1], I_asymp = Incidence[1]*(1-p_symp)/p_symp , R = 0)
const init = (S = N-Incidence[1], E = 0, I_symp = Incidence[1], I_asymp = 0 , R = 0);
const initfa = [Float64.(values(init))...];

## Dagen för "jobba hemma"
## 16 Mars 2020 = 76
## Previously a part of RSS.
@everywhere const t_b = dateToDay(Dates.Date("2020-03-15"));

## Independent functions
Aka. functions that are not reliant on local variables in `Estimate_function_Stockholm`.

Beta, aka. the infectivity is approximated as the following:

$\beta(t, \delta, \epsilon, \theta) = \theta\bigg(\frac{1-\delta}{1+e^{\epsilon*(-(t-t_b))}} + \delta\bigg)$

For some reason, Julia is quite fond of distinguishing the difference between keyword, and non-keyword arguments, hence the multiple definitions.

The basic reproductive number $R_0$ is normally calculated as the following, but here we also have corrections for the different infectivity of asymptomatic patients.

$R_0 = \frac{\beta}{\gamma}$

These functions only accept Float64 for performance reasons. Not even sure if this makes a difference, but one can change Float64 to Real if one wants to be able to hand other numbers to these.

In [8]:
## Function to create guesses for the optimisation
## The range of the guesses can be changed, 
## these are good for the specific dates and parameter combinations of p_symp and p_lower_inf
@everywhere function Guesses()::Vector{Float64}
    u_d = runif(1, 0.05, 0.6)[1] # guess for delta 
    u_e = runif(1,-0.6, 0)[1]    # guess for epsilon
    u_t = runif(1, 0, 15)[1]     # guess for theta
    return [u_d, u_e, u_t];
    end;

## The time-dependent infectivity rate
@everywhere function beta_decrease(t::Float64, delta::Float64, epsilon::Float64, theta::Float64)::Float64
    global t_b;
    return ((1-delta)/(1+exp(epsilon*(-(t-t_b)))) + delta)* theta;
    end;

# Fallback. Only to be used in the main thread.
# TODO check if this improves performance. The compiler is rather smart, so it may be able to handle
#      having the main function only have ::Real as arguments.
function beta_decrease(t::Real, delta::Real, epsilon::Real, theta::Real)::Float64
    return beta_decrease(Float64.([t, delta, epsilon, theta])...);
    end;

function beta_inverse(gamma::Real, delta::Real, epsilon::Real, theta::Real; planb::T = -Inf)::Union{T, Float64} where T <: Real
    global t_b; # dagen för jobba hemma
    local a::Float64 = ((1-delta)/((gamma/theta) - delta))-1;
    return (a > 0) ? -(log(a)/epsilon)+t_b : planb;
    end;

## Main function
Note that the means of optimization differs from the R implementation, as there are parts of that, specifically the things stored in the `conl` variable that I do not know how to implement with the Optim.jl package.

In [9]:
function Estimate_function_Stockholm(;
        p_symp::Real = 0.5,
        p_lower_inf::Real = 0.5,
        gammaD::Real = gammaD_value,
        eta::Real = eta_value,
        iter::Int = 50,
    )

    global N, Incidence, Datum, Dag, dayatmonth, dayatyear, Namedate, Opt_par_names;

    ## The SEIR model.
    function model!(du::Vector{Float64}, state::Vector{Float64}, parameters::Vector{Float64}, time::Float64)
        # S       <- state[1] # susceptibles
        # E       <- state[2] # latent/exposed but not infectious
        # I_symp  <- state[3] # infected who get reported
        # I_asymp <- state[4] # infected who remain non-reported
        # R       <- state[5] # recovered/immune
        
        # No need to specify types for these vars. The compiler can infer it, as long as the arg types are non-ambiguous
        # @views does not speed up this to any measurable extent.
        local S, E, I_symp, I_asymp, R = state;
        local delta, epsilon, theta = parameters;

        du[1] = -beta_decrease(time, delta, epsilon, theta) * S * I_symp/N - p_lower_inf*beta_decrease(time, delta, epsilon, theta) * S * I_asymp/N
        du[2] = beta_decrease(time, delta, epsilon, theta) * S * I_symp/N + p_lower_inf*beta_decrease(time, delta, epsilon, theta) * S * I_asymp/N - eta*E
        du[3] = p_symp * eta * E      - gammaD * I_symp
        du[4] = (1 - p_symp)* eta * E - gammaD * I_asymp
        du[5] = gammaD * (I_symp + I_asymp)
        end;

    function RSS(parameters::Vector{Float64})
        
        # Beta_max = theta. Beta_min = theta*delta. param order = delta, epsilon, theta
        # One could argue that we should check that epsilon is negative as well.
        # if the infectivity is negative, throw away guess
        if(parameters[2] > 0 || parameters[3] < 0 || parameters[1] < 0)
            return 10^12;
        else
            # choose tolerance atol and rtol
            local out = solve(ODEProblem(model!, initfa, DagspanF, parameters), Tsit5(), reltol=Rtol, abstol=Atol);

            # I tried making this more elegant, but it self-referenced when using repeat, which caused everything to become fit_I_asymp
            #local fit_S::Vector{Float64} = Vector{Float64}(undef, length(Dag));
            local fit_E::Vector{Float64} = Vector{Float64}(undef, length(Dag));
            #local fit_I_symp::Vector{Float64} = Vector{Float64}(undef, length(Dag));
            #local fit_I_asymp::Vector{Float64} = Vector{Float64}(undef, length(Dag));
            local fitted_incidence::Vector{Float64} = Vector{Float64}(undef, length(Dag));

            for (i, d) in enumerate(Dag)
                fit_E[i] = out(d)[2];
                # Not used, hence removed for SPEED.
                #fit_S[i] = out(d)[1];
                #fit_I_symp[i] = out(d)[3];
                #fit_I_asymp[i] = out(d)[4];
                end;
            
            fitted_incidence = eta * p_symp * fit_E;
            # Old, faulty version. Does not take incubation into consideration.
            #fitted_incidence = beta_decrease.(Dag, delta = parameters[1], epsilon = parameters[2],  theta = parameters[3]) .* fit_S .* fit_I_symp ./ N;

            return sum((Incidence - fitted_incidence).^ 2);
            #return DataFrame(T=Dag, S=fit_S, Is=fit_I_symp, Ia=fit_I_asymp);
            end;
        end;
    
    ## The time-dependent basic reproductive number
    function Basic_repr(t::Real, delta::Real, epsilon::Real, theta::Real, gamma::Real)::Float64
        return p_symp * beta_decrease(t, delta, epsilon, theta) / gamma + (1 - p_symp) * p_lower_inf * beta_decrease(t, delta, epsilon, theta) / gamma;
        end;
    
    # I don't know how to properly adjust the options to resemble the original code.
    # Specifically I don't know how feature scaling works with Optim.jl
    local conl::Optim.Options, Opt, optAlg, beta; 
    conl = Optim.Options(iterations = 1000, g_abstol = Atol, g_reltol = Rtol);
    Opt    = (x_converged = false,);
    optAlg = NelderMead();

    Opt = optimize(RSS, Guesses(), optAlg, conl);
    
    #@ Run until convergence. Should perhaps be multi-threaded.
    while(!(Opt.x_converged || Opt.f_converged || Opt.g_converged))
        Opt = optimize(RSS, Guesses(), optAlg, conl);
    end;
    
    local futures = Vector{Future}(undef, iter);
    # Brute-force approach.
    for i = 1:iter
        futures[i] = @spawnat :any try
            return optimize(RSS, Guesses(), optAlg, conl);
            catch; #catch e; return e; for debugging
            return false
        end;
    end;
    
    for i = 1:iter
        Opt2 = fetch(futures[i]);
        if(Opt2 != false && (Opt2.x_converged || Opt2.f_converged || Opt2.g_converged))
            if(Opt2.minimum < Opt.minimum && Opt2.minimizer[3] > 0 && Opt2.minimizer[1] > 0 && Opt2.minimizer[2] < 0)
                Opt = Opt2;
            end;
        end;
    end;
    
    return (
        Optimisation = Opt,
        SEIR_model = model!,
        RSS = RSS,
        Basic_reproduction = Basic_repr
    );
end;

In [15]:
# Performance check
# It may make sense to run these once, as it allows Julia to precompile these functions
# First run usually takes about 30 seconds (on an i5 6500, 4 Threads).
@time Estimate_function_Stockholm(p_symp = 0.0127, p_lower_inf = 1, iter = 20);

  2.006486 seconds (1.14 M allocations: 110.889 MiB, 0.57% gc time)


# Analysis

In [16]:
## These are also definied earlier

# Rate of leaving incubation
eta_value    = 1/5.1;
# Rate of recovery
gammaD_value = 1/5;

## Tolerance for ode and optimisation. 
Atol = 1e-8;
Rtol = 1e-10;

In [19]:
# Analysis  p_symp_use <- 0.0127
# Analysis  p_lower_inf_use <- 1, 0.55, 0.11
p_symp_use      = 0.0127
p_asymp_use     = 1 - p_symp_use
p_lower_inf_use = 3

# These are sadly both defined as local vars and here, global vars. Watch out.
eta = eta_value;
gammaD = gammaD_value;

Est_par_model = Estimate_function_Stockholm(p_symp = p_symp_use, p_lower_inf = p_lower_inf_use);

In [20]:
Est = Est_par_model.Optimisation;
Opt_par = (;zip(Symbol.(Opt_par_names), values(Est.minimizer))...);
Basic_repr = Est_par_model.Basic_reproduction;
RSS_value = Est.minimum;

H         = hessian(Est_par_model.RSS, [values(Opt_par)...]);
sigest    = sqrt(RSS_value/(length(Incidence)-3));
NeginvH2  = inv((1/(2*sigest^2)) * H);
sdParams  = sqrt.(Diagonal(NeginvH2).diag);

#println("Est = ", Est)
println("Opt_par = ", Opt_par)
println("sdParams = ", sdParams)
println("RSS_value = ", RSS_value)
print("Hessian:")
# Rounding just for the sake of printing, as it allows for easier comparison to the R code
Int64.(round.(H))

Opt_par = (delta = 0.12056865201721918, epsilon = -0.24803756766811513, theta = 0.6593373715066015)
sdParams = [0.01444396111272198, 0.04554580605791349, 0.006047801747364164]
RSS_value = 28899.017403598948
Hessian:

3×3 Array{Int64,2}:
 166496196  33785436  190806730
  33785436   7619595   35648710
 190806730  35648710  261988940

## Prevalence check
Look at the results, and compare with prevalence 27th March to 3rd April.

Basically this is done in order to verify how reasonable the estimate is. Folkhälsomyndiheten made a [cross-sectional prevalence study](https://www.folkhalsomyndigheten.se/publicerat-material/publikationsarkiv/f/forekomsten-av-covid-19-i-region-stockholm-26-mars3-april-2020/) during this time interval, and concluded that the prevalence at that point was likely 2.5% (95% CI &rarr; 1.4-4.1%). I'm not completely sure whether this takes the sensitivity and specificity of the test into account.

In [21]:
t = (Dag[1]):(Dag[length(Dag)]+14+11) # time in days
t2 = Float64.((t.start, t.stop)); # same as t, but in a format that the ode solver can deal with

# The following code looks very different as compared to the R code. 
# This is because the ode solver returns what would be the equivalent of the solution function.
# We therefore have to extract the values for discrete days in the loop below.
# fit <- data.frame(ode(y = init, times = t, func = SEIR_model , parms = Opt_par))

fit   = solve(ODEProblem(Est_par_model.SEIR_model, initfa, t2, [Opt_par...]), Tsit5());

fit_S       = Vector{Float64}(undef, length(t));
fit_E       = Vector{Float64}(undef, length(t));
fit_I_symp  = Vector{Float64}(undef, length(t));
fit_I_asymp = Vector{Float64}(undef, length(t));
fit_R       = Vector{Float64}(undef, length(t));
fit_I       = Vector{Float64}(undef, length(t));
fit_I_E     = Vector{Float64}(undef, length(t));
fit_cum_inf = Vector{Float64}(undef, length(t));

for (i, d) in enumerate(t)
    fit_S[i] = fit(d)[1];
    fit_E[i] = fit(d)[2];
    fit_I_symp[i] = fit(d)[3];
    fit_I_asymp[i] = fit(d)[4];
    fit_R[i] = fit(d)[5];
    fit_I = fit_I_symp + fit_I_asymp
    fit_I_E = fit_E + fit_I
    fit_cum_inf = N .- fit_S
end;


## The mean prevalence same days as the Hälsorapport Stockholmsstudien (27th March to 3rd April)
Smittsamma  = fit_I_symp + fit_I_asymp #+ fit_E
SmittsammaF =  Smittsamma[40:47]
println("Medelprevalens 27/3-3/4: ", round(mean(SmittsammaF/N)*100, digits = 3), "%")

## Look at the estimated reported cases and fitted

fitted_incidence = p_symp_use * fit_E * eta;
#fitted_incidence_non_report = (1 - p_symp_use) * fit_E * eta

Medelprevalens 27/3-3/4: 2.519%


## Initial plotting

In [23]:
# Plotting engine
#GR();
plotly(); 

In [24]:
plot(Datum, Incidence, label = "Reported cases", marker = (:circle, :white), line = (:black));
plot!(dayToDate.(t), fitted_incidence, label = "Fitted reported cases", line = (2, 0.7, :blue))
plot!(legend = :topleft, xlabel = "Day")

<!DOCTYPE html>
 
 
 Plots.jl

In [26]:
## Look at the estimated infectivity and basic reproductive number
## 1 jan, 1 feb  osv
dayatyear_march_april = [1, 32, 61, 61 + 31, 61 + 31 + 30, 61 + 31 + 30 + 31];
NameDateMarchApril = dayToDate.(dayatyear_march_april);
# par(mfrow = c(1,2), mar = c(6.1, 4.1, 6.1, 5.1)) # Not translated
plot(dayToDate.(0:150),
    Basic_repr.(0:150, Opt_par..., gammaD),
    ylabel = "R0(t)",
    title  = "Estimated reproductive number",
    label = "R0",
    legend = false,
    line = (1.5, :black)
    );
p1 = vline!([Datum[1], Datum[end]], line = (:dash, :gray))

plot(dayToDate.(0:150),
    beta_decrease.(0:150, Opt_par...),#type="l", ylab="Infectivity",lwd=2, 
    title = "Estimated infectivity",
    ylabel = "Infectivity, symptomatics",
    label = "Beta(t)",
    legend = false,
    line = (1.5, :black)
)
p2 = vline!([Datum[1], Datum[end]], line = (:dash, :gray));
plot(p1, p2, size = (800,400))

<!DOCTYPE html>
 
 
 Plots.jl

## Calculate results to save in tables
In the original code, they denote `delta` as `p`. Note that the minescule differences between this simulation and the original simulation start to add up as we do more and more calculations. These values can deviate as much as about 0.1 from the values in the R simulation, due to a slightly different `Opt_par`.

In [27]:
## To create bootstrap CI
CI_level_05 = 0.025

delta_high    = quantile(Normal(Opt_par[1], sdParams[1]), 1-CI_level_05);
epsilon_high  = quantile(Normal(Opt_par[2], sdParams[2]), 1-CI_level_05);
theta_high    = quantile(Normal(Opt_par[3], sdParams[3]), 1-CI_level_05);

delta_low     = max(0, quantile(Normal(Opt_par[1], sdParams[1]), CI_level_05));
epsilon_low   = quantile(Normal(Opt_par[2], sdParams[2]), CI_level_05);
theta_low     = quantile(Normal(Opt_par[3], sdParams[3]), CI_level_05);

In [28]:
delta_v   = rand(Normal(Opt_par[1], sdParams[1]), 1000);
delta_v   = [max(0, d) for d in delta_v];
epsilon_v = rand(Normal(Opt_par[2], sdParams[2]), 1000);
theta_v   = rand(Normal(Opt_par[3], sdParams[3]), 1000);

# Doesn't have to be initialized, but it's more efficient.
# List comprehension is also an option.
R0_v_Dag1 = Vector{Float64}(undef, length(delta_v));
R0_v_DagSista = Vector{Float64}(undef, length(delta_v));

for i in 1:length(delta_v)
    R0_v_Dag1[i]      = Basic_repr(Dag[1], delta_v[i], epsilon_v[i], theta_v[i], gammaD) 
    R0_v_DagSista[i]  = Basic_repr(Dag[end], delta_v[i], epsilon_v[i], theta_v[i], gammaD) 
end;

println("CRI-R0_v_Dag1: ", CRI(R0_v_Dag1, level= 0.95))
println("CRI-R0_v_DagSista: ", CRI(R0_v_DagSista, level= 0.95))

R0_low  = [CRI_95_low(R0_v_Dag1), CRI_95_low(R0_v_DagSista)];
R0_high = [CRI_95_up(R0_v_Dag1), CRI_95_up(R0_v_DagSista)];

R0_Mean = Basic_repr.(Dag, Opt_par..., gammaD);

println("R0_Mean[1]: ", R0_Mean[1])
println("R0_Mean[end]: ", R0_Mean[end])

CRI-R0_v_Dag1: (9.58457889895805, 9.969876524495882)
CRI-R0_v_DagSista: (0.9221343834529308, 1.4830434031710633)
R0_Mean[1]: 9.795690742498987
R0_Mean[end]: 1.1959581610257335


### Save estimated parameters and their SE

In [30]:
res_param = [round.((p_asymp_use, p_lower_inf_use), digits = 3)..., round(mean(SmittsammaF / N), digits = 5), round.((RSS_value, Opt_par[1], sdParams[1], Opt_par[2], sdParams[2], Opt_par[3], sdParams[3]), digits = 3)...];
# Julia does not allow duplicate names for NamedTuples.
res_param_names = ["p_0", "q_0","27 mars - 3 april", "RSS" ,"delta", "s.e.", "epsilon", "s.e.", "theta", "s.e."]

CIdelta   = "[" * join(round.([delta_low, delta_high],digits = 3), ",")   * "]";
CIepsilon = "[" * join(round.([epsilon_low, epsilon_high],digits = 3), ",") * "]";
CItheta   = "[" * join(round.([theta_low, theta_high],digits = 3), ",")   * "]";

CI_param = ["", "", "", "", CIdelta, "", CIepsilon, "", CItheta, ""];

df_res = DataFrame(Matrix{Any}(undef, 0, 10), Symbol.(res_param_names), makeunique=true);
push!(df_res, res_param);
push!(df_res, CI_param);

df_res |> save(join(["Results/Tables-Julia/","Res_para_p_non-reported_",p_asymp_use,"_infect_",p_lower_inf_use,".xlsx"]));

### Save results with 31 days forecast

In [330]:
# This was already calculated at an earlier stage...
t = (Dag[1]):(Dag[length(Dag)]+14+11) # time in days
t2 = Float64.((t.start, t.stop)); # same as t, but in a format that the ode solver can deal with

# The approach is somewhat different to that of the R code, due to a different library.
fit         = solve(ODEProblem(Est_par_model.SEIR_model, initfa, t2, [Opt_par...]), Tsit5());
fit_S       = Vector{Float64}(undef, length(t));
fit_E       = Vector{Float64}(undef, length(t));
fit_I_symp  = Vector{Float64}(undef, length(t));
fit_I_asymp = Vector{Float64}(undef, length(t));
fit_R       = Vector{Float64}(undef, length(t));
fit_I       = Vector{Float64}(undef, length(t));
fit_I_E     = Vector{Float64}(undef, length(t));
fit_cum_inf = Vector{Float64}(undef, length(t));

for (i, d) in enumerate(t)
    fit_S[i] = fit(d)[1];
    fit_E[i] = fit(d)[2];
    fit_I_symp[i] = fit(d)[3];
    fit_I_asymp[i] = fit(d)[4];
    fit_R[i] = fit(d)[5];
    fit_I = fit_I_symp + fit_I_asymp
    fit_I_E = fit_E + fit_I
    fit_cum_inf = N .- fit_S
end;


fitted_incidence = p_symp_use * fit_E * eta;
fitted_incidence_non_report = (1 - p_symp_use) * fit_E * eta;

Cum_Inf_inc = DataFrame(
    Cumulative = fit_cum_inf,
    Incidence_reported = fitted_incidence,
    Incidence_non_reported = fitted_incidence_non_report,
    Datum = dayToDate.(t)
);

Cum_Inf_inc = hcat(
    DataFrame(time = t, S = fit_S, E = fit_E, I_symp = fit_I_symp, I_asymp = fit_I_asymp, R = fit_R),
    Cum_Inf_inc
);

# Saved as csv here, instead of txt in the original code.
Cum_Inf_inc |> save(join(["Results/Tables-Julia/","Raw_data_fitted_model", "_para_p_asymp", p_asymp_use, "infect", p_lower_inf_use, ".csv"]));

### Now calculate bootstrap CI's

In [272]:
fit_S_v       = Matrix{Float64}(undef, length(delta_v), length(t));
fit_E_v       = Matrix{Float64}(undef, length(delta_v), length(t));
fit_I_symp_v  = Matrix{Float64}(undef, length(delta_v), length(t));
fit_I_asymp_v = Matrix{Float64}(undef, length(delta_v), length(t));
Fit_I_v       = Matrix{Float64}(undef, length(delta_v), length(t));
fit_cum_inf_v = Matrix{Float64}(undef, length(delta_v), length(t));

fitted_incidence_v = Matrix{Float64}(undef, length(delta_v), length(t));
effective_reprod_v = Matrix{Float64}(undef, length(delta_v), length(t));

In [313]:
for i = 1:length(delta_v)
    Opt_parDummy = (delta = delta_v[i], epsilon = epsilon_v[i], theta = theta_v[i]);
    fitDummy     = solve(ODEProblem(Est_par_model.SEIR_model, initfa, t2, [Opt_parDummy...]), Tsit5());
    
    for (r, d) in enumerate(t)
        fit_S_v[i, r] = fitDummy(d)[1];
        fit_E_v[i, r] = fitDummy(d)[2];
        fit_I_symp_v[i, r] = fitDummy(d)[3];
        fit_I_asymp_v[i, r] = fitDummy(d)[4];
        Fit_I_v[i, r] = fitDummy(d)[3] + fitDummy(d)[4];
        fit_cum_inf_v[i, r] = N - fitDummy(d)[1];
    end;

    fitted_incidence_v[i, :] = p_symp_use *  fit_E_v[i, :] * eta;
    #fitted_incidence.v[i,] <- beta(fitDummy[,1], delta = Opt_parDummy[1], epsilon = Opt_parDummy[2], theta = Opt_parDummy[3]) * fitDummy[ , 2] *  fitDummy[ , 4]/N 
    effective_reprod_v[i, :] = @. Basic_repr(t, Opt_parDummy..., gammaD) * fit_S_v[i, :] / N;
end;

In [377]:
cum_inf_mean       = mean(fit_cum_inf_v, dims = 1);
cum_inf_median     = median(fit_cum_inf_v, dims = 1);
cum_inf_95_up_CRI  = [CRI_95_up(col) for col in eachcol(fit_cum_inf_v)];
cum_inf_95_low_CRI = [CRI_95_low(col) for col in eachcol(fit_cum_inf_v)];

fit_I_mean       = mean(Fit_I_v, dims = 1);
fit_I_median     = median(Fit_I_v, dims = 1);
fit_I_95_up_CRI  = [CRI_95_up(col) for col in eachcol(Fit_I_v)];
fit_I_95_low_CRI = [CRI_95_low(col) for col in eachcol(Fit_I_v)];

fitted_Incidence_mean       = mean(fitted_incidence_v, dims = 1);
fitted_Incidence_median     = median(fitted_incidence_v, dims = 1);
fitted_Incidence_95_up_CRI  = [CRI_95_up(col) for col in eachcol(fitted_incidence_v)];
fitted_Incidence_95_low_CRI = [CRI_95_low(col) for col in eachcol(fitted_incidence_v)];

effective_reprod_mean       = mean(effective_reprod_v, dims = 1);
effective_reprod_median     = median(effective_reprod_v, dims = 1);
effective_reprod_95_up_CRI  = [CRI_95_up(col) for col in eachcol(effective_reprod_v)];
effective_reprod_95_low_CRI = [CRI_95_low(col) for col in eachcol(effective_reprod_v)];

# Cummulative number of infected until 2020-04-11 and until 2020-05-01, with their 95% CI
# 2020-04-11 = dag 102
# 2020-05-01 = dag 122

println("Dag 102: ", dayToDate(102));
println("Dag 122: ", dayToDate(122));

println.(vcat([[
fit_cum_inf[t.==d][1],
cum_inf_95_low_CRI[t.==d][1],
cum_inf_95_up_CRI[t.==d][1],

fit_cum_inf[t.==d][1]/N,
cum_inf_95_low_CRI[t.==d][1]/N,
cum_inf_95_up_CRI[t.==d][1]/N,
""] for d in [102, 122]]...));

maxdagen = dayToDate(t[fit_I .== maximum(fit_I)][1]);
minDag   = dayToDate(t[fit_I_95_low_CRI .== maximum(fit_I_95_low_CRI)][1]);
maxDag   = dayToDate(t[fit_I_95_up_CRI .== maximum(fit_I_95_up_CRI)][1]);

println.([maxdagen, minDag, maxDag]);

Dag 102: 2020-04-11
Dag 122: 2020-05-01
411475.323593837
277420.7251722587
617214.4710857661
0.17328560089020528
0.11683086276231651
0.2599290270096507

647286.6908547024
379495.25197532144
1.0622427715307705e6
0.27259341384881447
0.15981775577491375
0.44734487441021265

2020-04-11
2020-04-04
2020-04-19


### Save estimated R0 and their uncertainty

In [416]:
res_R0       = round.([p_asymp_use, p_lower_inf_use, R0_Mean[1], R0_Mean[length(Dag)], effective_reprod_mean[length(Dag)]], digits = 3);
res_R0_names = ["p_0", "q_0", "R0(start)", "R0(end)", "Re(end)"];
CI_R01       = join(["[",round(R0_low[1], digits=3), ", ", round(R0_high[1], digits=3),"]"]);
CI_ROend     = join(["[",round(R0_low[2], digits=3), ", ", round(R0_high[2], digits=3),"]"]);
CI_Reend     = join(["[",round(effective_reprod_95_low_CRI[length(Dag)],digits=3), ", ", round(effective_reprod_95_up_CRI[length(Dag)],digits=3),"]"]);
CIR0         = ["", "", CI_R01, CI_ROend,CI_Reend];

df_resR0 = DataFrame(Matrix{Any}(undef, 0, 5), Symbol.(res_R0_names), makeunique=true);
push!(df_resR0, res_R0);
push!(df_resR0, CIR0);

df_resR0 |> save(join(["Results/Tables-Julia/","J-Res_R0_p_non-reported_", p_asymp_use, "_infect_", p_lower_inf_use, ".xlsx"]));

### Save estimated days and their CI
TODO